I don't like the way numpy vector operations work, so I will implement my own, based on numpy element operations and total. All of these will work on stacks of vectors, where the first index indicates the vector component. If you want a 1D group of vectors, you want a 2D numpy array, where each column is a vector. 

As is customary for doing numerical work in interpreted languages, we want to be able to do operations on as large of arrays as possible without using an interpreted loop. In numpy as in IDL, we do this by doing array operations.

Frequently in order to have as large of an array as possible, we wil *stack* together vectors. We mostly use column vectors, so a 1D stack of vectors is a 2D array, where each column is one vector. A 1D stack of 2D matrices is a 3D array. We can imagine higher-dimension stacks easily -- if you have an nD stack of m-rank tensor objects, you need a (n+m)D numpy array. However, numpy itself doesn't have any concept of stacks and ranks, so you have to manually keep track of which array index is for the stack and which is for the object.

One inconvenient thing about array indexing in numpy is that it seems to be hard to make consistent. A 1D array of course only has one index. A row vector is a 2D array of size (1,n) while a column vector (most of the time we use column vectors) has size (n,1). Numpy doesn't make it easy to compose 1D vectors, row vectors, and/or column vectors, so they will usually have to be explicitly resized.

Further, if you have a stack of column vectors, it ends up being in the form (vector component, stack index). This is fine, because it exactly matches the form of a matrix. This is especially convenient when we want to transform a stack of vectors $[\mathbf{X}]=\begin{bmatrix}\vec{x}_0 & \vec{x}_1 & ... & \vec{x}_{n-1}\end{bmatrix}$ with a matrix $[\mathbf{A}]$ all at once: Just do a matrix multiply $[\mathbf{A}][\mathbf{X}]$.

Where things get inconsistent is higher dimensional stacks of vectors, or stacks of matrices. For a 1D stack of matrices, the indexes are (stack index, row index, column index). For a 2D stack of column vectors, we would have (stack1 index, vector component, stack0 index) (to keep things consistent with matrices). So, for vectors, the first stack index is the last index, but for matrices, all stack indexes are before the row and column indexes. For higher dimensional stacks of vectors, the first stack index is the last index, but all the other stack indexes come before the vector component index. This is mandated by how the `@` operator (implemented by [`numpy.matmul`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html)) treats indexes.

For a 1D stack of vectors with a total array shape of (n_comp,n_stack), we say the shape of the stack is (n_stack,). For a 2D stack of shape (n_stack0,n_comp,n_stack1), the stack shape is (n_stack0,n_stack1). For higher order stacks (n_stack0,n_stack1,...,n_stackn-1,n_comp,n_stackn) the stack shape is (n_stack0,n_stack1,n_stackn-1,n_stackn). Functions `vdot()`, `vlength()`, and `vangle()` which naturally take vector arguments and return scalars will return a stack of scalars, IE if you pass an nD stack of vectors (a numpy (n+1)D array), you will get a stack of scalars, IE a numpy nD array.

All of the complexity in these functions is just with dealing with stacks.

In [ ]:
import numpy as np
def vindex(v):
    if len(v.shape)>2:
        return len(v.shape)-2
    return 0

def vncomp(v):
    return v.shape[vindex(v)]

def vcomplimit(v,n):
    """
    Return a stack of vectors with the same shape as the input stack, but only
    including the first n vector components.
    :param v: input vector. Must have at least n components
    :param n: Number of vector components to keep
    :return:
    """
    if vindex(v)==0:
        return v[:n,...]
    else:
        return v[...,:n,:]

def vdot(a,b,array=False):
    """
    Dot product of two vectors or stack of vectors
    :param a: (nD stack of) Vector(s) for first dot product operand
    :param b: (nD stack of) Vector(s) for second dot product operand
    :param array: If true and passed a single vector for each operand, return a
        numpy 1D array result. Otherwise you will get a scalar result if you pass
        in single vector operands. No effect if passed stacks as either operand
    :return: Dot product(s) of inputs
    This uses numpy broadcasting to calculate the result, so the operands do not
    have to be the same size, just broadcast-compatible. In this case, the result
    may be larger than either input.
    
    If one input has more components than the other, the result will be equivalent
    to the result of the shorter input having the same number of components, all
    of which are zero. Equivalently, the result is equivalent to the longer input
    being truncated to match the length of the shorter input. Note that this only
    applies to the vector component -- all other axes of a stack are subject to 
    numpy broadcast rules.
    """
    n=np.min((vncomp(a),vncomp(b)))
    c=vcomplimit(a,n)*vcomplimit(b,n)
    result=np.sum(c,axis=vindex(c))
    if result.size==1 and not array:
        result=result.ravel()[0]
    if np.isscalar(result) and array:
        result=np.array([result])
    return result


def vlength(v,array=False):
    """
    Compute the length of a vector as the square root of the vector's dot product with itself
    :param v: (Stack of) Vector(s) to compute the length of
    :param array: Passed to vdot
    If true and passed a single vector, return a numpy 1D array result. Otherwise
                  you will get a scalar result if you pass a single vector.
    """
    return np.sqrt(vdot(v,v,array))

def vangle(a,b,array=False):
    """
    Compute the angle between two vectors
    :param a: (stack of) first  vector operand(s)
    :param b: (stack of) second vector operand(s)
    :param array: Passed to vdot and vlength
    :return: Angle(s) between two (stacks of) vectors in radians
    """
    return np.arccos(vdot(a,b,array)/vlength(a,array)/vlength(b,array))

def vcomp(v,l=None,minlen=None,maxlen=None):
    """
    Break a vector into components. an nD stack of m-element vectors will return a tuple with up to m elements,
    each of which will be an nD stack of scalars
    :param v: nD stack of m-element vectors, An nD stack of m-element vectors, a numpy (n+1)D array with shape
        (n_stack0,n_stack1,...,n_stackn-2,m,n_stackn-1)
    :param minlen: If passed, this will pad out the returned vector components with zero scalars
        such that the returned tuple has minlen components. We do zero scalars rather than zero arrays
        of the same size as the other components to save memory, since a scalar is compatible by
        broadcasting with an array of any size.
    :param maxlen: If passed, this will restrict the returned vector components to the given
        size, even if the input vector has more components.
    :return: A tuple. Each element is a vector component. Vector components pulled from the vector will be
        an nD stack of scalars, a numpy nD array with shape (n_stack0,n_stack1,...,n_stackn-2,n_stackn-1).
        Vector components which are made up will be scalar zeros.
    Note: If you pass maxlen<minlen, the result is still well-defined, since the maxlen is used first,
          then the minlen. If you pass a vector with m=4, a minlen of 7, and a maxlen of 2, you will get
          a result with the first two components of the vector, followed by 5 zeros. I'm not sure if this
          is useful, but there it is.
    """
    if maxlen is None and l is not None:
        maxlen=l
    if minlen is None and l is not None:
        minlen=l
    ndStack=len(v.shape) > 2
    efflen=v.shape[-2 if ndStack else 0]
    if maxlen is not None and maxlen<efflen:
        efflen=maxlen
    result=tuple([v[...,i,:] if ndStack else v[i,...] for i in range(efflen)])
    if minlen is not None and minlen>efflen:
        result=result+(0,)*(minlen-efflen)
    return result

def vcross(a,b):
    """
    Compute the three-dimensional cross-product of two vectors or stack of vectors
    :param a: (nD stack of) Vector(s) for first cross product operand
    :param b: (nD stack of) Vector(s) for second cross product operand
    :return: Cross product(s) of inputs
    This uses numpy broadcasting to calculate the result, so the operands do not
    have to be the same size, just broadcast-compatible. In this case, the result
    may be larger than either input.
    If either of the input vectors have fewer than three components, the extra components
    are made up and assumed to be zero. The result will always have three components.
    """
    (ax,ay,az)=vcomp(a,minlen=3,maxlen=3)
    (bx,by,bz)=vcomp(b,minlen=3,maxlen=3)
    cx=ay*bz-az*by
    cy=az*bx-ax*bz
    cz=ax*by-ay*bx
    if len(cx.shape)>=2:
        axis=-2
    else:
        axis=0
    result=np.stack([cx,cy,cz],axis=axis)
    return result



In [ ]:
print("1D row vectors")
a=np.array([1,2,3])
b=np.array([1,1,1])
ab=vdot(a,b)
print("a =",a)
print("b =",b)
print("ab=",ab," Should be 6")

In [ ]:
print("1D row vectors, mismatched length")
a=np.array([1,2,3])
b=np.array([1,1])
ab=vdot(a,b)
print("a =",a)
print("b =",b)
print("ab=",ab," Should be 3")

In [ ]:
print("1D row vectors, array=True")
a=np.array([1,2,3])
b=np.array([1,1,1])
ab=vdot(a,b,array=True)
print("a =",a)
print("b =",b)
print("ab=",ab," Should be [6]")

In [ ]:
print("1D column vectors")
a=np.array([[1],[2],[3]])
b=np.array([[1],[1],[1]])
ab=vdot(a,b)
print("a =",a)
print("b =",b)
print("ab=",ab," Should be 6")

In [ ]:
print("1D stack of column vectors")
a=np.array([[1,1],[2,2],[3,3]])
b=np.array([[1,1],[1,1],[1,1]])
ab=vdot(a,b)
print("a =",a)
print("b =",b)
print("ab=",ab, " Should be [6,6]")

In [ ]:
print("(2,2) stack of 3-element column vectors")
a=np.array([1,2,3])
a=np.stack((a,a),axis=1)
a=np.stack((a,a))
print("a =",a,a.shape)
b=np.array([1,1,1])
b=np.stack((b,b),axis=1)
b=np.stack((b,b))
print("b =",b,b.shape)
ab=vdot(a,b)
print("ab=",ab, " Should be 2D array [[6,6],[6,6]]",ab.shape)

In [ ]:
print("2-element cross product")
a=np.array([[1],[0]])
print("a =",a)
b=np.array([[0],[1]])
print("b =",b)
ab=vcross(a,b)
print("ab=",ab," Should be column vector [[0],[0],[1]]",ab.shape)

In [ ]:
print("2-element stack cross product")
a=np.array([[1,1],[0,0]])
print("a =",a)
b=np.array([[0,0],[1,1]])
print("b =",b)
ab=vcross(a,b)
print("ab=",ab," Should be 1D stack of column vectors [[0,0],[0,0],[1,1]]",ab.shape)

In [ ]:
print("3-element stack cross product")
a=np.array([[1,1],[0,0],[2,2]])
b=np.array([[0,0],[1,1],[2,2]])
ab=vcross(a,b)
print("a =",a,a.shape)
print("b =",b,b.shape)
print("ab=",ab," Shape should be (3,2)",ab.shape)

In [ ]:
print("Cross product of (2,2) stack of 3-element column vectors")
a=np.array([1,2,3])
a=np.stack((a,a),axis=1)
a=np.stack((a,a))
print("a =",a,a.shape)
b=np.array([1,1,1])
b=np.stack((b,b),axis=1)
b=np.stack((b,b))
print("b =",b,b.shape)
ab=vcross(a,b)
print("ab=",ab," Shape should be (2,3,2)",ab.shape)

In [ ]:
print("Cross product of row vectors")
a=np.array([1,0,2])
b=np.array([0,1,2])
ab=vcross(a,b)
print("a =",a,a.shape)
print("b =",b,b.shape)
print("ab=",ab," Shape should be (3,)",ab.shape)